In [1]:
#pip install pandas

In [2]:
#pip install ksql

In [3]:
import logging
from ksql import KSQLAPI
import pandas as pd
import datetime
import time


In [4]:
# 카프카에 저장된 데이터를 추가, 조회 가능한 KSQLDB 접속
# API 형식으로 카프카의 토픽(테이블)에 데이터를 추가 및 조회 가능한 객체 생성
client = KSQLAPI('http://localhost:8089', timeout= None)

In [5]:
result = client.ksql('show topics;')

In [6]:
result

[{'@type': 'kafka_topics',
  'statementText': 'show topics;',
  'topics': [{'name': 'car_location_topic', 'replicaInfo': [1]},
   {'name': 'default_ksql_processing_log', 'replicaInfo': [1]}],
  'warnings': []}]

In [7]:
# 버스 위치 정보가 저장된 location.csv 읽기

location_df = pd.read_csv("location.csv")

In [8]:
location_df.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [9]:
location_df.loc[0]

id                     210457
date               2019-10-29
route_id            405136001
vh_id                 7997025
route_nm                360-1
now_latitude        33.457724
now_longitude      126.554014
now_station              제대마을
now_arrive_time           07시
distance                333.0
next_station            제대아파트
next_latitude       33.458783
next_longitude     126.557353
Name: 0, dtype: object

In [10]:
#카프카에 전송할 데이터 수
count = len(location_df)
count

91774

## 각 버스들의 위치를 `카프카(Kafka)`로 전송

In [11]:
index = 0

while True:
    if index >= count:
        index = 0

    now = datetime.datetime.now()
    print(f"현재 시각 : {now}")

    date = now.strftime("%Y-%m-%d")
    print(f"현재 날짜 리턴 : {date}")
    
    now_arrive_time = now.hour
    weekday = now.weekday()
    print(f"현재 요일 : {weekday}")

    location_row = location_df.loc[index]

    route_id = location_df.loc[index,"route_id"]
    vh_id = location_df.loc[index,"vh_id"]
    route_nm = location_df.loc[index,"route_nm"]
    now_latitude = round(location_df.loc[index,"now_latitude"],6)
    now_longitude = round(location_df.loc[index,"now_longitude"],6)
    now_station = location_df.loc[index,"now_station"]
    distance = location_df.loc[index,"distance"]
    next_latitude = round(location_df.loc[index,"next_latitude"],6)
    next_longitude = round(location_df.loc[index,"next_longitude"],6)
    next_station = location_df.loc[index,"next_station"]
    
    # 현재 날짜와 시간을 정수로 id 저장
    id = int(now.timestamp())

    insert_query = f"""INSERT INTO car_location_topic (
                    id
                    ,route_id
                    ,vh_id
                    ,route_nm
                    ,now_latitude
                    ,now_longitude
                    ,now_station
                    ,now_arrive_time
                    ,distance
                    ,next_station
                    ,next_latitude
                    ,next_longitude
                    ,weekday
                    ,date
                    ) VALUES ({id},'{route_id}','{vh_id}'
                            ,'{route_nm}',{now_latitude}
                            ,{now_longitude},'{now_station}'
                            ,{now_arrive_time},{distance}
                            ,'{next_station}',{next_latitude}
                            ,{next_longitude},{weekday}
                            ,'{date}'
                            );"""
    print(f"insert query :\n{insert_query}")

    try:
        client.ksql(insert_query) # KSQLDB에 현재 위치 추가
    except Exception as e:
        print(f"Exception : \n{e}")

    # 30초 대기, 주기
    time.sleep(30)

    index +=1
    print("="*100)

현재 시각 : 2023-10-17 00:44:21.144142
현재 날짜 리턴 : 2023-10-17
현재 요일 : 1
inser query :
INSERT INTO car_location_topic (
                    id
                    ,route_id
                    ,vh_id
                    ,route_nm
                    ,now_latitude
                    ,now_longitude
                    ,now_station
                    ,now_arrive_time
                    ,distance
                    ,next_station
                    ,next_latitude
                    ,next_longitude
                    ,weekday
                    ,date
                    ) VALUES (1697471061,'405136001','7997025'
                            ,'360-1',33.457724
                            ,126.554014,'제대마을'
                            ,0,333.0
                            ,'제대아파트',33.458783
                            ,126.557353,1
                            ,'2023-10-17'
                            );
Exception : 
list index out of range
현재 시각 : 2023-10-17 00:44:51.348576
현재 날짜 리턴 : 2023-10-